# Trade accounting

Performs the Borin and Mancini (2019) exports decomposition. It loads the MRIO tables stored in `data/mrio/`. Results are saved as `ta.parquet` in `data/`.

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

## Setup

In [2]:
# input = 'adb-mrio.parquet'
# years = np.arange(2017, 2022+1)
# output = 'ta'
# version = None

input = 'adb-mrio62.parquet'
years = np.append(2000, np.arange(2007, 2022+1))
output = 'ta62'
version = None

# input = 'adb-mrio62-const.parquet'
# years = np.arange(2007, 2022+1)
# output = 'ta62-const'
# version = None

# input = 'adb-mrio_jun2023.parquet'
# years = np.arange(2020, 2022+1)
# output = 'ta'
# version = 'jun2023'

sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

# G = 73      # Number of countries + ROW
G = 63      
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

Note that the country index `s` used henceforth corresponds to the MRIO country indices, which start at 1 and not 0.

In [3]:
ta = pd.DataFrame()
ta_es = pd.DataFrame()
ta_os = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        # Country level

        Exports = np.sum(subset(E, s, -s), axis=0)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = subset(v, s) @ subset(Bnots, s, s)
        VB_FC = subset(v, -s) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s),
            'breakdown': 'none', 'i': 0, 'i5': 0, 'i15': 0,
            'Exports': Exports,
            'DAVAX1': DAVAX1, 'DAVAX2': DAVAX2,
            'REX1': REX1, 'REX2': REX2, 'REX3': REX3,
            'REF1': REF1, 'REF2': REF2,
            'FVA': FVA, 'PDC1': PDC1, 'PDC2': PDC2
        })
        ta = pd.concat([ta, ta_s], ignore_index=True)

        # Breakdown by export sectors

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        ta_es_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'es',
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_es = pd.concat([ta_es, ta_es_s], ignore_index=True)

        # Breakdown by origin sectors

        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_os_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'os',
            'i': np.tile(sectors['ind'], G-1), 'i5': np.tile(sectors['ind5'], G-1), 'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_os = pd.concat([ta_os, ta_os_s], ignore_index=True)

    print(f'{year} done')

2000 done
2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [4]:
ta = pd.concat([ta, ta_es], ignore_index=True)
ta = pd.concat([ta, ta_os], ignore_index=True)

if version is None:
    outputname = f'{output}.parquet'
else:
    outputname = f'{output}_{version}.parquet'

ta.to_parquet(f'../data/{outputname}', index=False)

In [5]:
ta

,t,s,r,breakdown,i,i5,i15,Exports,DAVAX1,DAVAX2,REX1,REX2,REX3,REF1,REF2,FVA,PDC1,PDC2
0,2000,1,2,none,0,0,0,61.954384,22.942670,18.511215,4.551384,7.186773,0.075608,0.031810,0.050342,8.542282,0.052635,0.009665
1,2000,1,3,none,0,0,0,345.467373,77.683622,65.132624,47.692980,98.011573,1.233481,0.186877,0.571957,54.541166,0.349343,0.063750
2,2000,1,4,none,0,0,0,5.710899,0.704824,3.756721,0.135933,0.419921,0.000430,0.000165,0.001811,0.686064,0.004286,0.000744
3,2000,1,5,none,0,0,0,557.705062,139.377316,301.742726,16.111656,29.814618,0.146017,0.159117,0.214613,69.648660,0.414622,0.075718
4,2000,1,6,none,0,0,0,1583.678810,414.752573,346.083941,221.548578,311.490586,6.508539,1.104626,1.980911,278.025606,1.845592,0.337857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714537,2022,63,62,os,31,5,13,3.901863,5.944805,16.393890,4.216543,10.588532,0.030973,2.026854,3.473147,24.908166,0.684051,0.275475
4714538,2022,63,62,os,32,5,14,13.296242,2.830025,9.175777,2.492160,5.494907,0.015715,1.276437,1.841624,9.698685,0.271178,0.101371
4714539,2022,63,62,os,33,5,14,6.409234,3.942084,8.267689,2.026097,4.977072,0.014512,0.979921,1.631905,4.038146,0.238738,0.044938
4714540,2022,63,62,os,34,5,15,58.110187,15.129124,31.324326,8.412939,19.432405,0.055648,4.163742,6.358925,28.251645,0.901745,0.306161
